In [ ]:
import pandas as pd
import json
import requests
import numpy as np
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from nltk.stem.snowball import SnowballStemmer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
from numpy import array
from sklearn.model_selection import train_test_split
import tensorflow as tf
import torch.optim as optim
from sklearn.model_selection import GridSearchCV
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.autograd import Variable
from sklearn import preprocessing
import tqdm
import time
import torch
import copy
from torch.utils.data import DataLoader, TensorDataset
import scipy.spatial.distance as ds

In [23]:
with open('/content/drive/MyDrive/case_2_data_for_members.json') as f:
    file_content = f.read()
    data = json.loads(file_content)


vector_of_vacancy = []
for i in range(len(data)-1):
  vector_of_vacancy.append(data[i]['vacancy'])

descriptions = []
for i in vector_of_vacancy:
  descriptions.append(i['description'])

In [25]:
#parce_employeers

empl = []

for i in range(len(data)-1):

    for j in data[i]["failed_resumes"]:
      s = ''
      if ("about" in j):
        if j['about'] is not None:
          s+=j['about'] + " "
      if ('experienceItem' in j):
        for k in j['experienceItem']:
          if ('description' in k):
            if k['description'] is not None:
              s += k['description'] + " "
      empl.append(s)

    for j in data[i]["confirmed_resumes"]:
      s = ''
      if ("about" in j):
        if j['about'] is not None:
          s+=j['about'] + " "
      if ('experienceItem' in j):
        for k in j['experienceItem']:
          if ('description' in k):
            if k['description'] is not None:
              s += k['description'] + " "
      empl.append(s)

In [26]:
def preprocess(arr):

  for i in range(len(arr)):

    arr[i] = re.sub(r'[.,!?\'\"();:]', '', arr[i])
    text = arr[i]
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('russian'))
    stemmer = SnowballStemmer("russian")
    filtered_tokens = [word for word in tokens if word not in stop_words]
    stemmed_words = [stemmer.stem(word) for word in filtered_tokens]
    arr[i] = stemmed_words

In [27]:
preprocess(empl)
preprocess(descriptions)

In [29]:
new_array = []
array_answer = []


for i in range(len(empl)):
  tmp = empl[i]
  k = 0
  while(k+100 < len(tmp)):
    new_array.append(tmp[k:k+100])
    array_answer.append(tmp[k+100])
    k+=1

for i in range(len(descriptions)):
  tmp = empl[i]
  k = 0
  while(k+100 < len(tmp)):
    new_array.append(tmp[k:k+100])
    array_answer.append(tmp[k+100])
    k+=1


In [ ]:
#tokens

sentences = new_array + array_answer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)

sequences = tokenizer.texts_to_sequences(sentences)

max_seq_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_seq_length, padding='post')

word_index = tokenizer.word_index
print("Word index:", word_index)

print("Encoded sequences:")
result = (padded_sequences - padded_sequences.mean()) / padded_sequences.std()


In [51]:
#data_for_train_test

result1 = result[:len(result)-108496]
result2 = result[len(result)-108496:]
res = []
for i in result2:
  res.append(i[0])

result2 = res

X = result1
y = result2
X_train, X_validation, y_train, y_validation = train_test_split(X, y, random_state=43, test_size=0.2)

In [77]:
#EarlyStopping


device = 'cpu'
print(f"Using device: {device}")

class EarlyStopping():

  def __init__(self, patience=5, min_delta=1e-3, restore_best_weights=True):
    self.patience = patience
    self.min_delta = min_delta
    self.restore_best_weights = restore_best_weights
    self.best_model = None
    self.best_loss = None
    self.counter = 0
    self.status = ""


  def __call__(self, model, val_loss):
    if self.best_loss == None:
      self.best_loss = val_loss
      self.best_model = copy.deepcopy(model)
    elif self.best_loss - val_loss > self.min_delta:
      self.best_loss = val_loss
      self.counter = 0
      self.best_model.load_state_dict(model.state_dict())
    elif self.best_loss - val_loss < self.min_delta:
      self.counter += 1
      if self.counter >= self.patience:
        self.status = f"Stopped on {self.counter}"
        if self.restore_best_weights:
          model.load_state_dict(self.best_model.state_dict())
        return True


    self.status = f"{self.counter}/{self.patience}"
    return False

Using device: cpu


In [ ]:
#model


class Net(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(Net, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):

        h0 = torch.zeros(self.num_layers, self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, self.hidden_size).to(x.device)
        out, (hn, cn) = self.lstm(x, (h0, c0))

        return out, hn, cn


X_train = torch.Tensor(X_train).float()
y_train = torch.Tensor(y_train).float()

X_validation = torch.Tensor(X_validation).float().to(device)
y_validation = torch.Tensor(y_validation).float().to(device)

batch_size = 16

dataset_train = TensorDataset(X_train, y_train)
dataloader_train = DataLoader(dataset_train,\
  batch_size=batch_size)

dataset_test = TensorDataset(X_validation, y_validation)
dataloader_test = DataLoader(dataset_test,\
  batch_size=batch_size)

model = Net(X.shape[1], 4, 2, 1).to(device)

loss_fn = nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

es = EarlyStopping()

epoch = 0
done = False
while epoch < 100 and not done:
  epoch += 1
  steps = list(enumerate(dataloader_train))
  pbar = tqdm.tqdm(steps)
  model.train()
  for i, (x_batch, y_batch) in pbar:
    y_batch_pred = model(x_batch.to(device))[0].flatten()
    loss = loss_fn(y_batch_pred[0], y_batch.to(device))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    loss, current = loss.item(), (i + 1)* len(x_batch)
    if i == len(steps)-1:
      model.eval()
      pred = model(X_validation)[0].flatten()
      vloss = loss_fn(pred[0], y_validation)
      if es(model,vloss): done = True
      pbar.set_description(f"Epoch: {epoch}, tloss: {loss}, vloss: {vloss:>7f}, EStop:[{es.status}]")
    else:
      pbar.set_description(f"Epoch: {epoch}, tloss {loss:}")

In [181]:
network = Net(X.shape[1], 4, 2, 1)
torch.save(network.state_dict(), '/content/model')

In [87]:
#predict and convert

def predict(model, input):
    model.eval()
    with torch.no_grad():
        predictions = model(input)
    return predictions

def convert_to_2D(arr):

  arr1 = np.zeros((1, len(arr)))
  for i in range(0, len(arr)):
      arr1[0][i] = arr[i]
  return arr1

In [143]:
#тесты

vacancy_test = data[len(data)-1]['vacancy']['description']
vacancy_test = [vacancy_test]
preprocess(vacancy_test)
failed = data[len(data)-1]["failed_resumes"]
good = data[len(data)-1]["confirmed_resumes"]

failed_array = []
for j in failed:
      s = ''
      if ("about" in j):
        if j['about'] is not None:
          s+=j['about'] + " "
      if ('experienceItem' in j):
        for k in j['experienceItem']:
          if ('description' in k):
            if k['description'] is not None:
              s += k['description'] + " "
      failed_array.append(s)


good_array = []
for j in good:
      s = ''
      if ("about" in j):
        if j['about'] is not None:
          s+=j['about'] + " "
      if ('experienceItem' in j):
        for k in j['experienceItem']:
          if ('description' in k):
            if k['description'] is not None:
              s += k['description'] + " "
      good_array.append(s)


preprocess(failed_array)
preprocess(good_array)

In [ ]:
#tokens

sentences = vacancy_test + good_array + failed_array

tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)

sequences = tokenizer.texts_to_sequences(sentences)

max_seq_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_seq_length, padding='post')

word_index = tokenizer.word_index
print("Word index:", word_index)


print("Encoded sequences:")
result = (padded_sequences - padded_sequences.mean()) / padded_sequences.std()


vac = result[0]
good = result[1:7]
bad = result[7:]

In [159]:
out, hidden_states_vac, cell_states = predict(model, torch.Tensor(convert_to_2D(vac[:100])).float())

In [173]:
for i in good:
  out, hidden_states_good, cell_states = predict(model, torch.Tensor(convert_to_2D(i[:100])).float())
  print('good', ds.cosine(hidden_states_vac[0], hidden_states_good[0]))

good 1.9806910753250122
good 0.9997750626207562
good 0.0040604472160339355
good 1.9959312081336975
good 0.01819556951522827
good 1.0010957774939016


In [175]:
for i in bad:
  out, hidden_states_bad, cell_states = predict(model, torch.Tensor(convert_to_2D(i[:100])).float())
  print('bad', ds.cosine(hidden_states_vac[0], hidden_states_bad[0]))

bad 1.23482283949852
bad 1.9955083131790161
bad 1.023731030523777
bad 0.0040604472160339355
bad 1.011991967447102
bad 1.995939314365387
bad 0.004060506820678711
bad 0.17646640539169312
bad 0.0040604472160339355
bad 1.995939552783966
bad 0.0040604472160339355
